In [165]:
import pandas as pd

def getReceiptMbs():
    df = pd.read_csv("data/1000i.csv",names = ['recpt_id','quantity','item'])
    
    # mbs: market baskets; maps the receipt number to a set of all the items purchased
    mbs = {}
    for row in df.values:
        item_id = row[2]
        r_id = row[0]

        if(r_id not in mbs):
            mbs[r_id] = set()

        mbs[r_id].add(item_id)
    
    return mbs

In [166]:
def getItemSets():
    df = pd.read_csv("data/1000i.csv",names = ['recpt_id','quantity','item'])
    
    return set(df['item'])

In [167]:
# Returns support of itemset
# Checks how many marketbaskets contain the itemset
def getSupport(itemset,mbs):
    count = 0
    for mb in mbs:
        if (itemset.issubset(mb)):
            count+=1
    return count/len(mbs)

In [168]:
"""
mbs: marketbaskets; map of receipt number to market basket
itemset: set of all items
minSup: minimum support number

return:
"""
def apriori(mbs, itemset, minSup):
    F = [] # list of F1, F2, ..., Fn
    F1 = [] # list of all item sets of length 1 where the support of the item set > minSup
    
    for item in itemset:            
        itemSup = getSupport(set([item]), mbs.values())
        if(itemSup >= minSup):
             F1.append(set([item]))
                
    F.append(F1)
    
    k = 1 #index to iterate F, eg. F[0] == F1
    while(len(F[k-1]) > 0):
        Ck = candidateGen(F[k-1], k-1) # candidate frequent itemsets of length k+1
        Fk = []
        
        for candidate in Ck:
            count = 0
            for mb in mbs.values():
                if(candidate.issubset(mb)):
                    count += 1

            if(count/len(mbs) >= minSup):
                Fk.append(candidate)
        
        F.append(Fk)   
        k += 1
                    
    return F

In [169]:
# Passing in arrray of itemsets of length k
# the size/length of the item sets K
# return: set of candidate frequent item sets of length k+1
def candidateGen(Fk, k):
    candidates = set()
    finalCandidates = set()
    
    #generate candidates of length k+1
    for itemset1 in Fk:
        for itemset2 in Fk:
            # check len(set) == k?
            union = itemset1.union(itemset2)
            if( (itemset1 is not itemset2) and (len(union) == len(itemset1) + 1) ):
                candidates.add(frozenset(union))
    
    #prune candidates
    for cand in candidates:
        isValid = True
        for item in cand:
            prunedCand = set([c for c in cand if c != item])
            if (prunedCand not in Fk):
                isValid = False
                continue;
        if (isValid):
            finalCandidates.add(cand)
            
    return finalCandidates 

In [170]:
def maximal(itemsets):
    all_itemsets = []
    maximal = []
    
    for itemset_list in itemsets:
        for itemset in itemset_list:
            all_itemsets.append(set(itemset))
    
    for itemset1 in all_itemsets:
        isMaximal = True
        for itemset2 in all_itemsets:
            if itemset1 is not itemset2 and itemset1.issubset(itemset2):
                isMaximal = False
        if isMaximal:
            maximal.append(itemset1)
        
                
    print("maximal: ", maximal)
    
    return maximal

In [171]:
def genRules(mbs, F, minConf):
    H1 = []
    
    for itemset in F:
        if len(itemset) < 2:
            continue;
        
        for item in itemset:
            conf = getSupport(itemset, mbs.values()) / getSupport(itemset - set([item]), mbs.values())
            if conf >= minConf:
                H1.append((itemset - set([item]), item))
        
    return H1   

In [172]:
def main():
    mbs = getReceiptMbs()
    itemsets = getItemSets()
    frequent_itemsets = apriori(mbs,itemsets, .04)
    maximal_itemsets = maximal(frequent_itemsets)
    rules = genRules(mbs, maximal_itemsets, .5)
    
    print("rules: ", rules)
    print("num rules: ", len(rules))

In [173]:
main()

maximal:  [{7}, {10}, {11}, {13}, {14}, {15}, {17}, {20}, {21}, {23}, {24}, {26}, {29}, {30}, {33}, {34}, {37}, {39}, {40}, {41}, {42}, {43}, {44}, {45}, {47}, {48}, {49}, {5, 22}, {27, 28}, {9, 4}, {1, 19}, {0, 2}, {18, 3}, {18, 35}, {16, 32}, {0, 46}, {36, 12, 31}]
rules:  [({22}, 5), ({5}, 22), ({28}, 27), ({27}, 28), ({4}, 9), ({9}, 4), ({19}, 1), ({2}, 0), ({3}, 18), ({35}, 18), ({18}, 35), ({32}, 16), ({46}, 0), ({0}, 46), ({12, 31}, 36), ({36, 31}, 12), ({36, 12}, 31)]
num rules:  17
